In [137]:
''' Extract data from file by the special keyword '''
import pandas as pd
import glob
import re
import numpy as np
import scipy.stats as stats
from scipy.stats import f_oneway

In [138]:
def read_log(file, regression_name):
    line_list = []
    with open(file) as f:
        while True:
            lines = f.readline()
            line_list.append(lines)
            if not lines:
                break
    # print(line_list)
    if regression_name == 'mlp':
        keyline = "INFO - Average testing results among all repeated 80-20 holdouts:\n"
    elif regression_name == 'svr':
        keyline = "Average testing results among all repeated 80-20 holdouts:\n"
    for i in range(len(line_list)):
        if line_list[i] == keyline:
            key_index = i
    srcc = line_list[key_index + 1]
    krcc = line_list[key_index + 2]
    plcc = line_list[key_index + 3]
    rmse = line_list[key_index + 4]
    return srcc, krcc, plcc, rmse

def get_value(coef, model_name, decimals):
    if model_name == 'mlp':
        pattern = r"INFO - (\w+): (\d+\.\d+) \(std: (\d+\.\d+)\)"
    elif model_name == 'svr':
        pattern = r"(\w+): (\d+\.\d+) \(std: (\d+\.\d+)\)"

    matches = re.findall(pattern, coef)
    for match in matches:
        metric, value, std_dev = match
        value = format(float(value), f'.{decimals}f')
        std_dev = format(float(std_dev), f'.{decimals}f')
        plusmius = u"\u00B1"
        coef_value = f"{value} ({plusmius}{std_dev})"
    return coef_value


In [139]:
def process_log_files(data_name, model_names, regression_name, log_path, decimals):
    df_results = pd.DataFrame(columns=['DATASET', 'MODEL', 'SRCC (±STD)', 'KRCC (±STD)', 'PLCC (±STD)', 'RMSE (±STD)'])
    data_list, model_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], [], []
    
    for model in model_names:
        log_file = f'{log_path}{data_name}_{model}_{regression_name}.log'
        try:
            srcc, krcc, plcc, rmse = read_log(log_file, regression_name)
            srcc = get_value(srcc, regression_name, decimals)
            krcc = get_value(krcc, regression_name, decimals)
            plcc = get_value(plcc, regression_name, decimals)
            rmse = get_value(rmse, regression_name, decimals)
            
            data_list.append(data_name)
            model_list.append(model)
            srcc_list.append(srcc)
            krcc_list.append(krcc)
            plcc_list.append(plcc)
            rmse_list.append(rmse)
        
        except FileNotFoundError:
            print(f"FileNotFoundError: No such file or directory for {model}")
        except Exception as e:
            print(f"Error occurred while processing {model}: {str(e)}")

    df_results['DATASET'] = data_list
    df_results['MODEL'] = model_list
    df_results['SRCC (±STD)'] = srcc_list
    df_results['KRCC (±STD)'] = krcc_list
    df_results['PLCC (±STD)'] = plcc_list
    df_results['RMSE (±STD)'] = rmse_list
    
    return df_results

Performance comparison of the evaluated NR-VQA models

In [140]:
data_name = 'cvd_2014'
model_names = ['brisque', 'vbliinds', 'tlvqm', 'videval', 'rapique']
regression_name = 'svr'
log_path = './reported_results/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results
# csv_name = f'./csv/{data_name}_svrlog.csv'
# df_results.to_csv(csv_name, index=None, encoding="UTF-8")
# xlsx_name = f'./save_csv/{data_name}_{regression_name}_log.xlsx'
# df_results.to_excel(xlsx_name, index=False, encoding="utf-8")

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,brisque,0.5553 (±0.0231),0.3895 (±0.0161),0.5527 (±0.0627),18.4752 (±1.0274)
1,cvd_2014,vbliinds,0.7306 (±0.0511),0.5393 (±0.0473),0.7853 (±0.0414),13.7267 (±1.2130)
2,cvd_2014,tlvqm,0.5399 (±0.0352),0.4006 (±0.0298),0.5785 (±0.0498),18.0832 (±0.7052)
3,cvd_2014,videval,0.7663 (±0.0214),0.5634 (±0.0221),0.8062 (±0.0216),13.1151 (±0.6579)
4,cvd_2014,rapique,0.8530 (±0.0301),0.6836 (±0.0357),0.8766 (±0.0326),10.6670 (±1.1939)


In [141]:
data_name = 'konvid_1k'
model_names = ['brisque', 'vbliinds', 'tlvqm', 'videval', 'rapique']
regression_name = 'svr'
log_path = './reported_results/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,brisque,0.6781 (±0.0083),0.4935 (±0.0091),0.6746 (±0.0069),0.4753 (±0.0040)
1,konvid_1k,vbliinds,0.7258 (±0.0115),0.5322 (±0.0109),0.7155 (±0.0107),0.4498 (±0.0071)
2,konvid_1k,tlvqm,0.7616 (±0.0067),0.5635 (±0.0067),0.7463 (±0.0065),0.4285 (±0.0047)
3,konvid_1k,videval,0.8073 (±0.0174),0.6036 (±0.0165),0.7923 (±0.0169),0.3928 (±0.0133)
4,konvid_1k,rapique,0.8219 (±0.0057),0.6264 (±0.0074),0.8191 (±0.0079),0.3693 (±0.0070)


In [142]:
data_name = 'live_vqc'
model_names = ['brisque', 'vbliinds', 'tlvqm', 'videval', 'rapique']
regression_name = 'svr'
log_path = './reported_results/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,live_vqc,brisque,0.6096 (±0.0133),0.4420 (±0.0127),0.6652 (±0.0176),12.7480 (±0.2697)
1,live_vqc,vbliinds,0.6925 (±0.0103),0.5046 (±0.0111),0.6872 (±0.0124),12.4031 (±0.1948)
2,live_vqc,tlvqm,0.8133 (±0.0104),0.6231 (±0.0105),0.7912 (±0.0148),10.4409 (±0.3123)
3,live_vqc,videval,0.7725 (±0.0093),0.5874 (±0.0080),0.7752 (±0.0211),10.7846 (±0.4082)
4,live_vqc,rapique,0.7328 (±0.0130),0.5418 (±0.0134),0.7520 (±0.0119),11.2538 (±0.2285)


In [143]:
data_name = 'youtube_ugc'
model_names = ['brisque', 'vbliinds', 'tlvqm', 'videval', 'rapique']
regression_name = 'svr'
log_path = './reported_results/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,youtube_ugc,brisque,0.3517 (±0.0165),0.2416 (±0.0119),0.3768 (±0.0200),0.6349 (±0.0056)
1,youtube_ugc,vbliinds,0.4783 (±0.0188),0.3396 (±0.0143),0.4967 (±0.0228),0.5949 (±0.0094)
2,youtube_ugc,tlvqm,0.6802 (±0.0183),0.4892 (±0.0168),0.6876 (±0.0207),0.4977 (±0.0132)
3,youtube_ugc,videval,0.7814 (±0.0049),0.5906 (±0.0052),0.7929 (±0.0038),0.4177 (±0.0033)
4,youtube_ugc,rapique,0.7994 (±0.0064),0.5989 (±0.0054),0.8300 (±0.0070),0.3823 (±0.0068)


vsfa performance

In [144]:
datasets = {
    'cvd_2014': "./reported_results/VSFA-CVD2014-EXP0_metrics.npy",
    'konvid_1k': "./reported_results/VSFA-KoNViD_1k-EXP0.npy",
    'live_vqc': "./reported_results/VSFA-LIVE-VQC-EXP0_metrics.npy",
    'youtube_ugc': "./reported_results/VSFA-YOUTUBE_UGC_ALL-EXP0.npy"
}
data_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], []
def load_metrics_npy(file_path, indices):
    data = np.load(file_path, allow_pickle=True)
    return [data[index] for index in indices]
for dataset_name, file_path in datasets.items():
    indices = [3, 4, 5, 6] if dataset_name in ['konvid_1k', 'youtube_ugc'] else [1, 2, 3, 4]
    srcc, krcc, plcc, rmse = load_metrics_npy(file_path, indices)
    data_list.append(dataset_name)
    srcc_list.append(srcc)
    krcc_list.append(krcc)
    plcc_list.append(plcc)
    rmse_list.append(rmse)

df_vsfa = pd.DataFrame({
    'DATASET': data_list,
    'MODEL': 'VSFA',
    'SRCC': srcc_list,
    'KRCC': krcc_list,
    'PLCC': plcc_list,
    'RMSE': rmse_list
}).round(4)
df_vsfa

,DATASET,MODEL,SRCC,KRCC,PLCC,RMSE
0,cvd_2014,VSFA,0.8825,0.7179,0.8784,9.8619
1,konvid_1k,VSFA,0.8067,0.6102,0.8182,0.4056
2,live_vqc,VSFA,0.5898,0.4120,0.5894,15.3245
3,youtube_ugc,VSFA,0.7857,0.5814,0.7808,0.4321


Performance on ReLax-VQA

In [145]:
data_name = 'cvd_2014'
model_names = ['relaxvqa']
regression_name = 'mlp'
log_path = './reported_results/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,relaxvqa,0.8643 (±0.0301),0.6960 (±0.0442),0.8895 (±0.0255),9.8185 (±1.0969)


In [146]:
data_name = 'konvid_1k'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,relaxvqa,0.8535 (±0.0218),0.6594 (±0.0232),0.8473 (±0.0214),0.3370 (±0.0185)


In [147]:
data_name = 'live_vqc'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,live_vqc,relaxvqa,0.7655 (±0.0378),0.5785 (±0.0365),0.8079 (±0.0352),9.8596 (±0.9099)


In [148]:
data_name = 'youtube_ugc'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,youtube_ugc,relaxvqa,0.8014 (±0.0196),0.6167 (±0.0193),0.8204 (±0.0186),0.3801 (±0.0177)


Ablation study on fragments

In [149]:
data_name = 'konvid_1k'
model_names = ['frame_diff_resnet50', 'optical_flow_resnet50', 'frame_diff_vit', 'optical_flow_vit', 'RF_frame_diff_resnet50', 'RF_optical_flow_resnet50', 'RF_frame_diff_vit', 'RF_optical_flow_vit',
               'MF_resnet50', 'MF_vit', 'MF_resnet50_vit']
regression_name = 'mlp'
log_path = './reported_results/ablation_study/fragmentation/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,frame_diff_resnet50,0.6526 (±0.0221),0.4654 (±0.0170),0.6525 (±0.0202),0.4889 (±0.0153)
1,konvid_1k,optical_flow_resnet50,0.5793 (±0.0392),0.4088 (±0.0298),0.5848 (±0.0423),0.5356 (±0.0233)
2,konvid_1k,frame_diff_vit,0.6384 (±0.0333),0.4633 (±0.0245),0.6454 (±0.0298),0.4890 (±0.0162)
3,konvid_1k,optical_flow_vit,0.5692 (±0.0376),0.4075 (±0.0293),0.5844 (±0.0385),0.5243 (±0.0191)
4,konvid_1k,RF_frame_diff_resnet50,0.7082 (±0.0345),0.5168 (±0.0301),0.7073 (±0.0345),0.4547 (±0.0185)
5,konvid_1k,RF_optical_flow_resnet50,0.4844 (±0.0504),0.3356 (±0.0369),0.4958 (±0.0518),0.5634 (±0.0213)
6,konvid_1k,RF_frame_diff_vit,0.7145 (±0.0297),0.5221 (±0.0265),0.7226 (±0.0281),0.4487 (±0.0153)
7,konvid_1k,RF_optical_flow_vit,0.5032 (±0.0495),0.3508 (±0.0360),0.5335 (±0.0582),0.5504 (±0.0224)
8,konvid_1k,MF_resnet50,0.6792 (±0.0424),0.4852 (±0.0342),0.6788 (±0.0440),0.4761 (±0.0254)
9,konvid_1k,MF_vit,0.7062 (±0.0298),0.5130 (±0.0270),0.7057 (±0.0296),0.4582 (±0.0175)


Ablation study on layer stack

In [150]:
data_name = 'konvid_1k'
model_names = ['pool_vgg16', 'pool_resnet50', 'pool_vit', 'layer_stack_vgg16', 'layer_stack_resnet50', 'pool_pool_vgg16_vit', 'pool_pool_resnet50_vit', 'layer_stack_pool_vgg16_vit', 'layer_stack_pool_resnet50_vit']
regression_name = 'mlp'
log_path = './reported_results/ablation_study/layer_stack/'
df_results = process_log_files(data_name, model_names, regression_name, log_path, decimals=4)
df_results

,DATASET,MODEL,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,pool_vgg16,0.6665 (±0.0370),0.4878 (±0.0313),0.6880 (±0.0328),0.4629 (±0.0188)
1,konvid_1k,pool_resnet50,0.6751 (±0.0313),0.4881 (±0.0257),0.7047 (±0.0292),0.4550 (±0.0162)
2,konvid_1k,pool_vit,0.6661 (±0.0396),0.4906 (±0.0332),0.7082 (±0.0356),0.4515 (±0.0199)
3,konvid_1k,layer_stack_vgg16,0.7513 (±0.0250),0.5626 (±0.0228),0.7700 (±0.0219),0.4108 (±0.0186)
4,konvid_1k,layer_stack_resnet50,0.7630 (±0.0239),0.5699 (±0.0219),0.7828 (±0.0224),0.3990 (±0.0162)
5,konvid_1k,pool_pool_vgg16_vit,0.7100 (±0.0273),0.5210 (±0.0235),0.7364 (±0.0247),0.4370 (±0.0153)
6,konvid_1k,pool_pool_resnet50_vit,0.7253 (±0.0259),0.5308 (±0.0223),0.7469 (±0.0263),0.4277 (±0.0151)
7,konvid_1k,layer_stack_pool_vgg16_vit,0.7650 (±0.0245),0.5742 (±0.0236),0.7827 (±0.0213),0.4042 (±0.0177)
8,konvid_1k,layer_stack_pool_resnet50_vit,0.7697 (±0.0230),0.5779 (±0.0213),0.7897 (±0.0209),0.3948 (±0.0152)


In [151]:
def process_finetune_csv(pattern, select_criteria, is_finetune):
    csv_files = glob.glob(pattern)
    combined_df = pd.DataFrame()
    if is_finetune == True:
        for file in csv_files:
            df = pd.read_csv(file)
            df['LAYER_NAME'] = 'layer_stack+pool'
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            df.to_csv(file, index=False)
        combined_df = combined_df.sort_values(by='DATASET')
        combined_df = combined_df.round(4)
        df_byrmse = combined_df[combined_df['SELECT_CRITERIA'] == 'byrmse']
        df_bykrcc = combined_df[combined_df['SELECT_CRITERIA'] == 'bykrcc']
        if len(df_byrmse) == len(df_bykrcc):
            combined_horizontal = pd.concat([df_byrmse.reset_index(drop=True), df_bykrcc.reset_index(drop=True)], axis=1)
            print(f"Performance with SELECT_CRITERIA = {select_criteria} on LSVQ trained model")
            return combined_horizontal
    else:
        for file in csv_files:
            df = pd.read_csv(file)
            df['LAYER_NAME'] = 'layer_stack+pool'
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            df.to_csv(file, index=False)
        combined_df = combined_df.sort_values(by='DATASET')
        combined_df = combined_df.round(4)
        return combined_df

with Fine-tuning on an LSVQ-trained model

In [152]:
log_path = './reported_results/fine_tune/'
pattern_rmse = log_path + "/*_relaxvqa_byrmse_finetune.csv"
result = process_finetune_csv(pattern_rmse, 'byrmse', True)
result

Performance with SELECT_CRITERIA = byrmse on LSVQ trained model


,DATASET,MODEL,LAYER_NAME,SRCC,KRCC,PLCC,RMSE,SELECT_CRITERIA,DATASET,MODEL,LAYER_NAME,SRCC,KRCC,PLCC,RMSE,SELECT_CRITERIA
0,cvd_2014,relaxvqa,layer_stack+pool,0.8974,0.7299,0.9294,8.1812,byrmse,cvd_2014,relaxvqa,layer_stack+pool,0.8899,0.7169,0.9124,9.0719,bykrcc
1,konvid_1k,relaxvqa,layer_stack+pool,0.8720,0.6881,0.8668,0.3211,byrmse,konvid_1k,relaxvqa,layer_stack+pool,0.8694,0.6834,0.8620,0.3263,bykrcc
2,live_vqc,relaxvqa,layer_stack+pool,0.8468,0.6649,0.8876,7.9869,byrmse,live_vqc,relaxvqa,layer_stack+pool,0.8367,0.6531,0.8726,8.4696,bykrcc
3,youtube_ugc,relaxvqa,layer_stack+pool,0.8469,0.6623,0.8652,0.3437,byrmse,youtube_ugc,relaxvqa,layer_stack+pool,0.8456,0.6562,0.8631,0.3462,bykrcc


In [153]:
# pattern_krcc = "./reported_results/old/*_relaxvqa_bykrcc_finetune.csv"
# result = process_finetune_csv(pattern_krcc, 'bykrcc', True)
# result

without Fine-tuning on an LSVQ-trained model

In [154]:
pattern_rmse_wo = log_path + "/*_relaxvqa_byrmse*_wo_finetune.csv"
result_wo_finetune = process_finetune_csv(pattern_rmse_wo, 'byrmse', False)
result_wo_finetune

,DATASET,MODEL,LAYER_NAME,SRCC,KRCC,PLCC,RMSE,SELECT_CRITERIA
1,cvd_2014,relaxvqa,layer_stack+pool,0.7845,0.5930,0.8336,12.2445,byrmse
0,konvid_1k,relaxvqa,layer_stack+pool,0.8312,0.6418,0.8427,0.3466,byrmse
2,live_vqc,relaxvqa,layer_stack+pool,0.7664,0.5812,0.8242,9.8201,byrmse
3,youtube_ugc,relaxvqa,layer_stack+pool,0.8104,0.6131,0.8354,0.3768,byrmse


In [155]:
# pattern_krcc_wo = "./reported_results/old/*_relaxvqa_bykrcc*_wo_finetune.csv"
# result = process_finetune_csv(pattern_krcc_wo, 'bykrcc', False)
# result_wo_finetune

In [156]:
import pandas as pd
from scipy.stats import levene
        # 'VSFA': f'./reported_results/ss_test/{dataset_name.upper()}_RAPIQUE_svr_median_results.csv',
def f_test(dataset_name):
    csv_files = {
        'BRISQUE': f'./reported_results/ss_test/{dataset_name.upper()}_BRISQUE_svr_median_results.csv',
        'V-BLIINDS': f'./reported_results/ss_test/{dataset_name.upper()}_VBLIINDS_svr_median_results.csv',
        'TLVQM': f'./reported_results/ss_test/{dataset_name.upper()}_TLVQM_svr_median_results.csv',
        'VIDEVAL': f'./reported_results/ss_test/{dataset_name.upper()}_VIDEVAL_svr_median_results.csv',
        'RAPIQUE': f'./reported_results/ss_test/{dataset_name.upper()}_RAPIQUE_svr_median_results.csv',
        'ReLaX-VQA': f'./reported_results/fine_tune/predicted_score/base/{dataset_name}_relaxvqa_byrmse.csv',    
        'ReLaX-VQA (w/o FT)': f'./reported_results/fine_tune/predicted_score/wo_finetune/{dataset_name}_relaxvqa_byrmse_predicted_score.csv',    
        'ReLaX-VQA (w/ FT)': f'./reported_results/fine_tune/predicted_score/finetune/{dataset_name}_relaxvqa_byrmse_predicted_score.csv',   
    }

    model_predictions = {}
    for model, file in csv_files.items():
        df = pd.read_csv(file)
        if model in ['BRISQUE', 'V-BLIINDS', 'TLVQM', 'VIDEVAL', 'RAPIQUE']:
            df = df.sort_values(by='y_test')
            model_predictions[model] = df['y_test_pred'].values
        else:
            df = df.sort_values(by='MOS')
            model_predictions[model] = df['y_test_pred'].values
    
    performance_metrics = {}
    for model, predictions in model_predictions.items():
        df = pd.read_csv(csv_files[model])
        if model in ['BRISQUE', 'V-BLIINDS', 'TLVQM', 'VIDEVAL', 'RAPIQUE']:
            df = df.sort_values(by='y_test')
            true_scores = df['y_test'].values
        else:
            df = df.sort_values(by='MOS')
            true_scores = df['MOS'].values

        srcc = pd.Series(predictions).corr(pd.Series(true_scores), method='spearman')
        plcc = pd.Series(predictions).corr(pd.Series(true_scores), method='pearson')
        performance_metrics[model] = {'SRCC': srcc, 'PLCC': plcc}

    stats_results = {}
    ref_model = 'ReLaX-VQA'
    ref_predictions = model_predictions[ref_model]
    ref_srcc = performance_metrics[ref_model]['SRCC']
    ref_plcc = performance_metrics[ref_model]['PLCC']

    for model, predictions in model_predictions.items():
        if model != ref_model:
            stat, p_value = levene(ref_predictions, predictions)
            if p_value < 0.05:
                model_srcc = performance_metrics[model]['SRCC']
                model_plcc = performance_metrics[model]['PLCC']
                if model_srcc > ref_srcc and model_plcc > ref_plcc:
                    y = 1
                else:
                    y = -1
            else:
                y = 0
            stats_results[model] = {'Levene_stat': stat, 'p_value': p_value, 'y': y}
    
    for model, result in stats_results.items():
        print(f"Model: {model}, Levene statistic: {result['Levene_stat']:.4f}, p-value: {result['p_value']:}, y: {result['y']} (1=better, -1=worse, 0=no difference)")

dataset_name = 'konvid_1k'
f_test(dataset_name)

Model: BRISQUE, Levene statistic: 26.0611, p-value: 4.783843384622389e-07, y: -1 (1=better, -1=worse, 0=no difference)
Model: V-BLIINDS, Levene statistic: 40.4563, p-value: 4.707796423944759e-10, y: -1 (1=better, -1=worse, 0=no difference)
Model: TLVQM, Levene statistic: 12.1645, p-value: 0.0005320762657629913, y: -1 (1=better, -1=worse, 0=no difference)
Model: VIDEVAL, Levene statistic: 11.0811, p-value: 0.0009394928717871182, y: -1 (1=better, -1=worse, 0=no difference)
Model: RAPIQUE, Levene statistic: 15.0879, p-value: 0.00011707612923417454, y: -1 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/o FT), Levene statistic: 0.6910, p-value: 0.40624637874254066, y: 0 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/ FT), Levene statistic: 384.4812, p-value: 2.9882849098132805e-63, y: 1 (1=better, -1=worse, 0=no difference)


In [157]:
dataset_name = 'cvd_2014'
f_test(dataset_name)

Model: BRISQUE, Levene statistic: 5.8915, p-value: 0.017161566924953884, y: -1 (1=better, -1=worse, 0=no difference)
Model: V-BLIINDS, Levene statistic: 6.6067, p-value: 0.011765655703063588, y: -1 (1=better, -1=worse, 0=no difference)
Model: TLVQM, Levene statistic: 25.9459, p-value: 1.8595873362110528e-06, y: -1 (1=better, -1=worse, 0=no difference)
Model: VIDEVAL, Levene statistic: 8.7815, p-value: 0.0038735440718641284, y: -1 (1=better, -1=worse, 0=no difference)
Model: RAPIQUE, Levene statistic: 6.4391, p-value: 0.012845841363931009, y: -1 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/o FT), Levene statistic: 0.7308, p-value: 0.3948589539076568, y: 0 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/ FT), Levene statistic: 0.2902, p-value: 0.5913834641624721, y: 0 (1=better, -1=worse, 0=no difference)


In [158]:
dataset_name = 'live_vqc'
f_test(dataset_name)

Model: BRISQUE, Levene statistic: 11.8775, p-value: 0.0006748580974470261, y: -1 (1=better, -1=worse, 0=no difference)
Model: V-BLIINDS, Levene statistic: 1.8981, p-value: 0.1696239713038948, y: 0 (1=better, -1=worse, 0=no difference)
Model: TLVQM, Levene statistic: 0.0671, p-value: 0.7958998566559031, y: 0 (1=better, -1=worse, 0=no difference)
Model: VIDEVAL, Levene statistic: 0.0137, p-value: 0.9068841428601186, y: 0 (1=better, -1=worse, 0=no difference)
Model: RAPIQUE, Levene statistic: 0.0833, p-value: 0.7731710122948385, y: 0 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/o FT), Levene statistic: 5.3744, p-value: 0.021304337433851846, y: 1 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/ FT), Levene statistic: 7.2406, p-value: 0.007645898457470841, y: 1 (1=better, -1=worse, 0=no difference)


In [159]:
dataset_name = 'youtube_ugc'
f_test(dataset_name)

Model: BRISQUE, Levene statistic: 119.6830, p-value: 8.389004730706275e-25, y: -1 (1=better, -1=worse, 0=no difference)
Model: V-BLIINDS, Levene statistic: 99.6782, p-value: 2.7065361557577903e-21, y: -1 (1=better, -1=worse, 0=no difference)
Model: TLVQM, Levene statistic: 5.9081, p-value: 0.015470166702581065, y: -1 (1=better, -1=worse, 0=no difference)
Model: VIDEVAL, Levene statistic: 5.3780, p-value: 0.020849634337515167, y: -1 (1=better, -1=worse, 0=no difference)
Model: RAPIQUE, Levene statistic: 10.0987, p-value: 0.0015883263468545986, y: -1 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/o FT), Levene statistic: 0.0880, p-value: 0.7669264214551041, y: 0 (1=better, -1=worse, 0=no difference)
Model: ReLaX-VQA (w/ FT), Levene statistic: 330.5777, p-value: 1.6688310177405526e-55, y: 1 (1=better, -1=worse, 0=no difference)


overall performance

In [160]:
# new
data = {
    'BRISQUE': {'SRCC': [0.5553, 0.6781, 0.6096, 0.3517], 'PLCC': [0.5527, 0.6746, 0.6652, 0.3768]},
    'V-BLIINDS': {'SRCC': [0.7306, 0.7258, 0.6925, 0.4783], 'PLCC': [0.7853, 0.7155, 0.6872, 0.4967]},
    'TLVQM': {'SRCC': [0.5399, 0.7616, 0.8133, 0.6802], 'PLCC': [0.5785, 0.7463, 0.7912, 0.6876]},
    'VIDEVAL': {'SRCC': [0.7663, 0.8073, 0.7725, 0.7814], 'PLCC': [0.8062, 0.7923, 0.7752, 0.7929]},
    'RAPIQUE': {'SRCC': [0.8530, 0.8219, 0.7328, 0.7994], 'PLCC': [0.8766, 0.8191, 0.7520, 0.8300]},
    'VSFA': {'SRCC': [0.8825, 0.8067, 0.5898, 0.7857], 'PLCC': [0.8784, 0.8182, 0.5894, 0.7808]},
    'ReLaX-VQA (ours)': {'SRCC': [0.8643, 0.8535, 0.7655, 0.8014], 'PLCC': [0.8895, 0.8473, 0.8079, 0.8204]},
    'ReLaX-VQA (w/o FT)': {'SRCC': [0.7845, 0.8312, 0.7664, 0.8104], 'PLCC': [0.8336, 0.8427, 0.8242, 0.8354]},
    'ReLaX-VQA (w/ FT)': {'SRCC': [0.8974, 0.8720, 0.8468, 0.8469], 'PLCC': [0.9294, 0.8668, 0.8876, 0.8652]},
}
# avg performance of each model for each dataset
avg_performance = {}
for model, metrics in data.items():
    avg_srcc = np.mean(metrics['SRCC'])
    avg_plcc = np.mean(metrics['PLCC'])
    avg_performance[model] = {'SRCC': avg_srcc, 'PLCC': avg_plcc}
df_avg_performance = pd.DataFrame.from_dict(avg_performance, orient='index').round(4)
df_avg_performance

,SRCC,PLCC
BRISQUE,0.5487,0.5673
V-BLIINDS,0.6568,0.6712
TLVQM,0.6988,0.7009
VIDEVAL,0.7819,0.7916
RAPIQUE,0.8018,0.8194
VSFA,0.7662,0.7667
ReLaX-VQA (ours),0.8212,0.8413
ReLaX-VQA (w/o FT),0.7981,0.8340
ReLaX-VQA (w/ FT),0.8658,0.8872
